### Generate Data
Generate a sBC count matrix for a dense grid of 100,000 beads using the Slide-seq Kidney dataset bead qualities downsampled to a median of 50 UMI/ 140 reads per bead.

In [4]:
import numpy as np
import pandas as pd
import anndata
import math

import sys
sys.path.append("../functions/")
import barcode_sampling
import utils

In [5]:
# Define parameters for the run
SEED = 42
RNG = np.random.default_rng(SEED)
NUM_PROC = 32

SATELLITES_PER_CM2 = 10**5
SIGMA_UM = 50 # Sigma for the satellite diffusion Gaussians in um
SAMPLE = 'kidney_140rpb'

BEADS_PER_CM2 = 10**6
AREA_CM2 = 0.1

BEADS = int(BEADS_PER_CM2 * AREA_CM2)
SATELLITES = int(SATELLITES_PER_CM2 * AREA_CM2)
n = np.floor(np.sqrt(BEADS))
# Standard deviation for the Gaussian modelling diffusion around a satellite
# Divide by n*10 to account for our simulation being on a unit square
SIGMA = SIGMA_UM/(n*10) 

# Factor by which satellites should extended past beads, this reduces edge effects from beads along the
# edge getting fewer reads
SATELLITE_EXT = 0.05

In [6]:
# Generate random colonies on a unit square and beads on a grid
y,x = np.meshgrid(np.arange(0,n), np.arange(0,n))
y = y.flatten()
x = x.flatten()
bead_coords = np.array([x/max(x), y/max(y)]).T
satellite_coords = RNG.random((int((SATELLITES)*((1+2*SATELLITE_EXT)**2)), 2))*(1 + 2*SATELLITE_EXT) - SATELLITE_EXT

beads_df = pd.DataFrame(data=bead_coords, columns=['x', 'y'])
satellites_df = pd.DataFrame(data=satellite_coords, columns=['x', 'y'])

In [7]:
# Load the bead-quality and barcode overlap distributions
reads_dist = pd.read_csv('../data/bead-quality-distributions/{}.csv'.format(SAMPLE), index_col=0)
overlap_dist = pd.read_csv('../data/barcode-distribution.csv', index_col=0)

# Generate the read matrix
satellite_counts = barcode_sampling.get_colony_counts(SIGMA, bead_coords, satellite_coords,
                                                     num_proc=NUM_PROC,
                                                     overlap_dist=overlap_dist['Dist'],
                                                     reads_dist=reads_dist,
                                                     chunksize=30000,
                                                     scaling=False)

100%|██████████| 99856/99856 [00:11<00:00, 8906.11it/s]


Matrix build time: 11.82694959640503


In [8]:
# Create and save the adata
adata = anndata.AnnData(X=satellite_counts, obs=beads_df, var=pd.DataFrame(index=range(satellite_counts.shape[1])), dtype=np.int32)
adata.obs['spatial_color'] = utils.radial_color(adata.obs.x, adata.obs.y, x0=0.5, y0=0.5)
adata.write_h5ad('../data/adata/{}_satellites={}_sigma={}.h5ad'.format(SAMPLE, SATELLITES_PER_CM2, SIGMA_UM))

/mnt/geofflab/jupyterhub/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'spatial_color' as categorical
